In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Introduction

#### I have tried many models and it is my best result with Tensorflow library i think. Many Thanks to Laurent Pourchot for his notebook [Less is more](https://www.kaggle.com/pourchot/only-one-hidden-layer-for-neural-network). All in All this competetition is not adding much because we are predicting noise behaviour so there will be again big shakeup in LB. But lets have fun :)

In [ ]:
!pip install tensorflow==2.5.0

In [ ]:
pip install -U tensorflow_decision_forests

In [ ]:
pip install wurlitzer

In [ ]:
tf.__version__

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, Reshape, BatchNormalization, Add,Dropout, Concatenate, Average, Embedding,  Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer, QuantileTransformer, OneHotEncoder, KBinsDiscretizer
from sklearn.decomposition import PCA

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.cluster import KMeans
from tensorflow import keras
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tensorflow.keras import backend as K
import tensorflow_decision_forests as tfdf
from wurlitzer import sys_pipes

In [ ]:
pd.set_option('display.max_rows',150)

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv')
X_test =  pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv')
train.pop('id')

y = train['loss']
train.pop('loss')

X_test.pop('id')
X=train

In [ ]:
X_train, X_val, y_train, y_val  = train_test_split(X,y,test_size=0.05,random_state=2021,stratify=y)

In [ ]:
# X_train_cat=X_train[['f1','f16','f27','f55','f86']]
# X_val_cat=X_val[['f1','f16','f27','f55','f86']]
# X_test_cat=X_test[['f1','f16','f27','f55','f86']]
# X_train.drop(['f1','f16','f27','f55','f86'], axis=1)
# X_val.drop(['f1','f16','f27','f55','f86'], axis=1)
# X_test.drop(['f1','f16','f27','f55','f86'], axis=1)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
qt = QuantileTransformer(n_quantiles=60, output_distribution='uniform')

X_train = qt.fit_transform(X_train)
X_val = qt.transform(X_val)
X_test = qt.transform(X_test)


## Binning the features in 60 categories

In [ ]:
bin = KBinsDiscretizer(n_bins=60, encode='ordinal',strategy='uniform')
X_train = bin.fit_transform(X_train)
X_val = bin.transform(X_val)
X_test = bin.transform(X_test)
# X = bin.fit_transform(X)
# X_test = bin.transform(X_test)

In [ ]:
# bin_cat = KBinsDiscretizer(n_bins=10, encode='ordinal',strategy='uniform')
# X_train_cat = bin_cat.fit_transform(X_train_cat)
# X_val_cat = bin_cat.transform(X_val_cat)
# X_test_cat = bin_cat.transform(X_test_cat)

In [ ]:
# X_train_cat.shape

## LESS IS MORE :) Thing i learned from Laurent Pourchot

In [ ]:

def RegModel(input_shape):

    

    X_input = Input(input_shape)
    X = Embedding (input_dim=60, output_dim=30)(X_input)
    X = Flatten()(X)
    X = Dropout(0.5)(X)

    ll = Dense(40, kernel_initializer=tf.keras.initializers.GlorotNormal(), activation='swish')(X)
#     X = BatchNormalization()(X)
    X = Dropout(0.5)(ll)
    X = Dense(1, kernel_initializer=tf.keras.initializers.GlorotNormal(),activation='swish', name='output2')(X)
    model = Model(inputs = X_input, outputs = X, name='RegModel')
    regmodel_wo_head = Model(inputs=model.inputs, outputs=ll)

    df_and_nn_model = tfdf.keras.GradientBoostedTreesModel(preprocessing=regmodel_wo_head,
                                                task=tfdf.keras.Task.REGRESSION,
                                                num_trees=300,
                                                max_depth=8,
                                                max_num_nodes=-1,
                                                min_examples=512,
                                                validation_ratio=0.5,
                                                l2_regularization=0.5,
                                                subsample=0.8,
                                                early_stopping='MIN_LOSS_FINAL',
                                                shrinkage=0.008,
                                                growing_strategy='BEST_FIRST_GLOBAL')
    return model, regmodel_wo_head , df_and_nn_model


In [ ]:
BATCH_SIZE=64
SHUFFLE_BUFFER_SIZE = 100
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,np.float32((y_train))))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val,np.float32((y_val))))
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
train_dataset.take(1)

### Note for Huber loss, the higher delta the more MSE the lower delte the more MAE, in 20th runs i found out that delta ~34-46 making best scores

# For 1 - fold predictions

In [ ]:
checkpoint_filepath = '/kaggle/working/ckpt1'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
regmodel, regmodel_wo_head, df_and_nn_model  = RegModel(X_train.shape[1:])

regmodel.compile(loss=tf.keras.losses.Huber(38), optimizer = keras.optimizers.Adam(learning_rate=0.0001), metrics=[tf.keras.metrics.RootMeanSquaredError()])
        
regmodel.fit(train_dataset,
        epochs = 10, 
        validation_data=val_dataset,
        callbacks=[model_checkpoint_callback])

In [ ]:
#regmodel.load_weights(checkpoint_filepath)
df_and_nn_model.compile(metrics=[tf.keras.metrics.RootMeanSquaredError()])
with sys_pipes():
    df_and_nn_model.fit(train_dataset, validation_data=val_dataset)

In [ ]:
main_pred = df_and_nn_model.predict(test_dataset)

In [ ]:
main_pred[:50]

In [ ]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')
sub.iloc[:,1]=main_pred
sub=sub.set_index('id')
sub.to_csv('noise_prediction.csv')